In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName("Formula 1").getOrCreate()

In [0]:
# writing code to get the data from azure Data lake gen 2 account into databricks

In [0]:
data_lake_name = "gen02sto1059"
data_lake_key = "NXYzHuDCK9n3ZRU1ccIctKX0X5PszwooNfFtjTc6NeQz1XX6PO2DJQ3OzRC91HJy9HMUeEmaTivW+AStdFeivw=="
container = "adlscon1"

In [0]:
spark.conf.set(f"fs.azure.account.key.{data_lake_name}.blob.core.windows.net", data_lake_key)

In [0]:
dbutils.fs.ls(f"wasbs://{container}@{data_lake_name}.blob.core.windows.net/")

[FileInfo(path='wasbs://adlscon1@gen02sto1059.blob.core.windows.net/circuits.csv', name='circuits.csv', size=10122, modificationTime=1709207443000),
 FileInfo(path='wasbs://adlscon1@gen02sto1059.blob.core.windows.net/constructors.json', name='constructors.json', size=30623, modificationTime=1709207443000),
 FileInfo(path='wasbs://adlscon1@gen02sto1059.blob.core.windows.net/drivers.json', name='drivers.json', size=181664, modificationTime=1709207444000),
 FileInfo(path='wasbs://adlscon1@gen02sto1059.blob.core.windows.net/lap_times_split_1.csv', name='lap_times_split_1.csv', size=36182, modificationTime=1709207419000),
 FileInfo(path='wasbs://adlscon1@gen02sto1059.blob.core.windows.net/pit_stops.json', name='pit_stops.json', size=9238, modificationTime=1709207443000),
 FileInfo(path='wasbs://adlscon1@gen02sto1059.blob.core.windows.net/qualifying_split_1.json', name='qualifying_split_1.json', size=4138, modificationTime=1709207419000),
 FileInfo(path='wasbs://adlscon1@gen02sto1059.blob.co

In [0]:
# loading the data into dataframe

In [0]:
df_races = spark.read.csv(f"wasbs://{container}@{data_lake_name}.blob.core.windows.net/races.csv", header=True, inferSchema=True)

In [0]:
df_races.show(7)

+------+----+-----+---------+--------------------+----------+--------+--------------------+
|raceId|year|round|circuitId|                name|      date|    time|                 url|
+------+----+-----+---------+--------------------+----------+--------+--------------------+
|     1|2009|    1|        1|Australian Grand ...|2009-03-29|06:00:00|http://en.wikiped...|
|     2|2009|    2|        2|Malaysian Grand Prix|2009-04-05|09:00:00|http://en.wikiped...|
|     3|2009|    3|       17|  Chinese Grand Prix|2009-04-19|07:00:00|http://en.wikiped...|
|     4|2009|    4|        3|  Bahrain Grand Prix|2009-04-26|12:00:00|http://en.wikiped...|
|     5|2009|    5|        4|  Spanish Grand Prix|2009-05-10|12:00:00|http://en.wikiped...|
|     6|2009|    6|        6|   Monaco Grand Prix|2009-05-24|12:00:00|http://en.wikiped...|
|     7|2009|    7|        5|  Turkish Grand Prix|2009-06-07|12:00:00|http://en.wikiped...|
+------+----+-----+---------+--------------------+----------+--------+----------

In [0]:
# Instead of explicitly specifying the full path every time to access the data, we can use mounting

In [0]:
dbutils.fs.mount(
    source=f"wasbs://{container}@{data_lake_name}.blob.core.windows.net/",
    mount_point="/mnt/files",
    extra_configs={f"fs.azure.account.key.{data_lake_name}.blob.core.windows.net": data_lake_key}
)

True

In [0]:
dbutils.fs.mounts()

[MountInfo(mountPoint='/mnt/storage', source='wasbs://con2@mycon2.blob.core.windows.net/', encryptionType=''),
 MountInfo(mountPoint='/databricks-datasets', source='databricks-datasets', encryptionType=''),
 MountInfo(mountPoint='/Volumes', source='UnityCatalogVolumes', encryptionType=''),
 MountInfo(mountPoint='/databricks/mlflow-tracking', source='databricks/mlflow-tracking', encryptionType=''),
 MountInfo(mountPoint='/databricks-results', source='databricks-results', encryptionType=''),
 MountInfo(mountPoint='/databricks/mlflow-registry', source='databricks/mlflow-registry', encryptionType=''),
 MountInfo(mountPoint='/mnt/files', source='wasbs://adlscon1@gen02sto1059.blob.core.windows.net/', encryptionType=''),
 MountInfo(mountPoint='/Volume', source='DbfsReserved', encryptionType=''),
 MountInfo(mountPoint='/volumes', source='DbfsReserved', encryptionType=''),
 MountInfo(mountPoint='/', source='DatabricksRoot', encryptionType=''),
 MountInfo(mountPoint='/volume', source='DbfsReserv

In [0]:
df_races = spark.read.csv("/mnt/files/races.csv", header=True, inferSchema=True)

In [0]:
df_races.show(10)

+------+----+-----+---------+--------------------+----------+--------+--------------------+
|raceId|year|round|circuitId|                name|      date|    time|                 url|
+------+----+-----+---------+--------------------+----------+--------+--------------------+
|     1|2009|    1|        1|Australian Grand ...|2009-03-29|06:00:00|http://en.wikiped...|
|     2|2009|    2|        2|Malaysian Grand Prix|2009-04-05|09:00:00|http://en.wikiped...|
|     3|2009|    3|       17|  Chinese Grand Prix|2009-04-19|07:00:00|http://en.wikiped...|
|     4|2009|    4|        3|  Bahrain Grand Prix|2009-04-26|12:00:00|http://en.wikiped...|
|     5|2009|    5|        4|  Spanish Grand Prix|2009-05-10|12:00:00|http://en.wikiped...|
|     6|2009|    6|        6|   Monaco Grand Prix|2009-05-24|12:00:00|http://en.wikiped...|
|     7|2009|    7|        5|  Turkish Grand Prix|2009-06-07|12:00:00|http://en.wikiped...|
|     8|2009|    8|        9|  British Grand Prix|2009-06-21|12:00:00|http://en.

In [0]:
df_races.printSchema()

root
 |-- raceId: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- round: integer (nullable = true)
 |-- circuitId: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- date: date (nullable = true)
 |-- time: string (nullable = true)
 |-- url: string (nullable = true)



In [0]:
df_circuits = spark.read.csv("/mnt/files/circuits.csv", header=True, inferSchema=True)

In [0]:
df_circuits.show(10)

+---------+--------------+--------------------+------------+---------+--------+--------+---+--------------------+
|circuitId|    circuitRef|                name|    location|  country|     lat|     lng|alt|                 url|
+---------+--------------+--------------------+------------+---------+--------+--------+---+--------------------+
|        1|   albert_park|Albert Park Grand...|   Melbourne|Australia|-37.8497| 144.968| 10|http://en.wikiped...|
|        2|        sepang|Sepang Internatio...|Kuala Lumpur| Malaysia| 2.76083| 101.738| 18|http://en.wikiped...|
|        3|       bahrain|Bahrain Internati...|      Sakhir|  Bahrain| 26.0325| 50.5106|  7|http://en.wikiped...|
|        4|     catalunya|Circuit de Barcel...|    Montmeló|    Spain|   41.57| 2.26111|109|http://en.wikiped...|
|        5|      istanbul|       Istanbul Park|    Istanbul|   Turkey| 40.9517|  29.405|130|http://en.wikiped...|
|        6|        monaco|   Circuit de Monaco| Monte-Carlo|   Monaco| 43.7347| 7.42056|

In [0]:
df_circuits = df_circuits.withColumnRenamed("circuitId", "circuit_id").withColumnRenamed("circuitRef", "circuit_ref").withColumnRenamed("name", "Name").withColumnRenamed("location", "Location").withColumnRenamed("country", "Country").withColumnRenamed("lat", 'Latitude').withColumnRenamed("lng", "Longitude").withColumnRenamed("alt", "Altitude").withColumnRenamed('url', "URL")

In [0]:
df_circuits.show(5)

+----------+-----------+--------------------+------------+---------+--------+---------+--------+--------------------+
|circuit_id|circuit_ref|                Name|    Location|  Country|Latitude|Longitude|Altitude|                 URL|
+----------+-----------+--------------------+------------+---------+--------+---------+--------+--------------------+
|         1|albert_park|Albert Park Grand...|   Melbourne|Australia|-37.8497|  144.968|      10|http://en.wikiped...|
|         2|     sepang|Sepang Internatio...|Kuala Lumpur| Malaysia| 2.76083|  101.738|      18|http://en.wikiped...|
|         3|    bahrain|Bahrain Internati...|      Sakhir|  Bahrain| 26.0325|  50.5106|       7|http://en.wikiped...|
|         4|  catalunya|Circuit de Barcel...|    Montmeló|    Spain|   41.57|  2.26111|     109|http://en.wikiped...|
|         5|   istanbul|       Istanbul Park|    Istanbul|   Turkey| 40.9517|   29.405|     130|http://en.wikiped...|
+----------+-----------+--------------------+-----------

In [0]:
# Final circuits dataframe
display(df_circuits)

circuit_id,circuit_ref,Name,Location,Country,Latitude,Longitude,Altitude,URL
1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,http://en.wikipedia.org/wiki/Melbourne_Grand_Prix_Circuit
2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.738,18,http://en.wikipedia.org/wiki/Sepang_International_Circuit
3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,7,http://en.wikipedia.org/wiki/Bahrain_International_Circuit
4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57,2.26111,109,http://en.wikipedia.org/wiki/Circuit_de_Barcelona-Catalunya
5,istanbul,Istanbul Park,Istanbul,Turkey,40.9517,29.405,130,http://en.wikipedia.org/wiki/Istanbul_Park
6,monaco,Circuit de Monaco,Monte-Carlo,Monaco,43.7347,7.42056,7,http://en.wikipedia.org/wiki/Circuit_de_Monaco
7,villeneuve,Circuit Gilles Villeneuve,Montreal,Canada,45.5,-73.5228,13,http://en.wikipedia.org/wiki/Circuit_Gilles_Villeneuve
8,magny_cours,Circuit de Nevers Magny-Cours,Magny Cours,France,46.8642,3.16361,228,http://en.wikipedia.org/wiki/Circuit_de_Nevers_Magny-Cours
9,silverstone,Silverstone Circuit,Silverstone,UK,52.0786,-1.01694,153,http://en.wikipedia.org/wiki/Silverstone_Circuit
10,hockenheimring,Hockenheimring,Hockenheim,Germany,49.3278,8.56583,103,http://en.wikipedia.org/wiki/Hockenheimring


In [0]:
# Joining circuits to races

In [0]:
df_races_circuits = df_races.join(df_circuits, df_races.circuitId == df_circuits.circuit_id, how="inner")

In [0]:
display(df_races_circuits)

raceId,year,round,circuitId,name,date,time,url,circuit_id,circuit_ref,Name,Location,Country,Latitude,Longitude,Altitude,URL
1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_Grand_Prix,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,http://en.wikipedia.org/wiki/Melbourne_Grand_Prix_Circuit
2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Grand_Prix,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.738,18,http://en.wikipedia.org/wiki/Sepang_International_Circuit
3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Grand_Prix,17,shanghai,Shanghai International Circuit,Shanghai,China,31.3389,121.22,5,http://en.wikipedia.org/wiki/Shanghai_International_Circuit
4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Grand_Prix,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,7,http://en.wikipedia.org/wiki/Bahrain_International_Circuit
5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Grand_Prix,4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57,2.26111,109,http://en.wikipedia.org/wiki/Circuit_de_Barcelona-Catalunya
6,2009,6,6,Monaco Grand Prix,2009-05-24,12:00:00,http://en.wikipedia.org/wiki/2009_Monaco_Grand_Prix,6,monaco,Circuit de Monaco,Monte-Carlo,Monaco,43.7347,7.42056,7,http://en.wikipedia.org/wiki/Circuit_de_Monaco
7,2009,7,5,Turkish Grand Prix,2009-06-07,12:00:00,http://en.wikipedia.org/wiki/2009_Turkish_Grand_Prix,5,istanbul,Istanbul Park,Istanbul,Turkey,40.9517,29.405,130,http://en.wikipedia.org/wiki/Istanbul_Park
8,2009,8,9,British Grand Prix,2009-06-21,12:00:00,http://en.wikipedia.org/wiki/2009_British_Grand_Prix,9,silverstone,Silverstone Circuit,Silverstone,UK,52.0786,-1.01694,153,http://en.wikipedia.org/wiki/Silverstone_Circuit
9,2009,9,20,German Grand Prix,2009-07-12,12:00:00,http://en.wikipedia.org/wiki/2009_German_Grand_Prix,20,nurburgring,Nürburgring,Nürburg,Germany,50.3356,6.9475,578,http://en.wikipedia.org/wiki/N%C3%BCrburgring
10,2009,10,11,Hungarian Grand Prix,2009-07-26,12:00:00,http://en.wikipedia.org/wiki/2009_Hungarian_Grand_Prix,11,hungaroring,Hungaroring,Budapest,Hungary,47.5789,19.2486,264,http://en.wikipedia.org/wiki/Hungaroring


In [0]:
df_results_json = spark.read.json("/mnt/files/results.json")

In [0]:
df_results = df_results_json.write.option("header","True").csv("/mnt/files/csv_results.csv")

In [0]:
display(df_results)

In [0]:
df_results = df_results_json.write.csv("/mnt/files/results.csv", header=True, mode="overwrite")

In [0]:
df_results_json.show()

+-------------+--------+----------+---------------+--------------+----+----+------------+------+------+--------+-------------+------------+------+----+--------+--------+-----------+
|constructorId|driverId|fastestLap|fastestLapSpeed|fastestLapTime|grid|laps|milliseconds|number|points|position|positionOrder|positionText|raceId|rank|resultId|statusId|       time|
+-------------+--------+----------+---------------+--------------+----+----+------------+------+------+--------+-------------+------------+------+----+--------+--------+-----------+
|            9|     830|        60|         227.96|      1:17.524|   3|  63|     7354598|    33|    25|       1|            1|           1|  1053|   2|   24986|       1|2:02:34.598|
|          131|       1|        60|        230.403|      1:16.702|   1|  63|     7376598|    44|    19|       2|            2|           2|  1053|   1|   24987|       1|    +22.000|
|            1|     846|        63|        225.819|      1:18.259|   7|  63|     7378300| 

In [0]:
df_results = spark.read.csv("/mnt/files/results.csv", header=True, inferSchema=True)

In [0]:
df_results.show(10)

+-------------+--------+----------+---------------+--------------+----+----+------------+------+------+--------+-------------+------------+------+----+--------+--------+-----------+
|constructorId|driverId|fastestLap|fastestLapSpeed|fastestLapTime|grid|laps|milliseconds|number|points|position|positionOrder|positionText|raceId|rank|resultId|statusId|       time|
+-------------+--------+----------+---------------+--------------+----+----+------------+------+------+--------+-------------+------------+------+----+--------+--------+-----------+
|            9|     830|        60|         227.96|      1:17.524|   3|  63|     7354598|    33|    25|       1|            1|           1|  1053|   2|   24986|       1|2:02:34.598|
|          131|       1|        60|        230.403|      1:16.702|   1|  63|     7376598|    44|    19|       2|            2|           2|  1053|   1|   24987|       1|    +22.000|
|            1|     846|        63|        225.819|      1:18.259|   7|  63|     7378300| 

In [0]:
df_drivers_json = spark.read.json("/mnt/files/drivers.json")

In [0]:
df_drivers_json.show(10)

+----+----------+--------+----------+--------------------+-----------+------+--------------------+
|code|       dob|driverId| driverRef|                name|nationality|number|                 url|
+----+----------+--------+----------+--------------------+-----------+------+--------------------+
| HAM|1985-01-07|       1|  hamilton|   {Lewis, Hamilton}|    British|    44|http://en.wikiped...|
| HEI|1977-05-10|       2|  heidfeld|    {Nick, Heidfeld}|     German|    \N|http://en.wikiped...|
| ROS|1985-06-27|       3|   rosberg|     {Nico, Rosberg}|     German|     6|http://en.wikiped...|
| ALO|1981-07-29|       4|    alonso|  {Fernando, Alonso}|    Spanish|    14|http://en.wikiped...|
| KOV|1981-10-19|       5|kovalainen|{Heikki, Kovalainen}|    Finnish|    \N|http://en.wikiped...|
| NAK|1985-01-11|       6|  nakajima|  {Kazuki, Nakajima}|   Japanese|    \N|http://en.wikiped...|
| BOU|1979-02-28|       7|  bourdais|{Sébastien, Bourd...|     French|    \N|http://en.wikiped...|
| RAI|1979

In [0]:
df_constructors = spark.read.json("/mnt/files/constructors.json")

In [0]:
df_constructors.show(10)

+-------------+--------------+-----------+-----------+--------------------+
|constructorId|constructorRef|       name|nationality|                 url|
+-------------+--------------+-----------+-----------+--------------------+
|            1|       mclaren|    McLaren|    British|http://en.wikiped...|
|            2|    bmw_sauber| BMW Sauber|     German|http://en.wikiped...|
|            3|      williams|   Williams|    British|http://en.wikiped...|
|            4|       renault|    Renault|     French|http://en.wikiped...|
|            5|    toro_rosso| Toro Rosso|    Italian|http://en.wikiped...|
|            6|       ferrari|    Ferrari|    Italian|http://en.wikiped...|
|            7|        toyota|     Toyota|   Japanese|http://en.wikiped...|
|            8|   super_aguri|Super Aguri|   Japanese|http://en.wikiped...|
|            9|      red_bull|   Red Bull|   Austrian|http://en.wikiped...|
|           10|   force_india|Force India|     Indian|http://en.wikiped...|
+-----------

In [0]:
%sql
create database if not exists formula_1

In [0]:
df_races.createOrReplaceTempView("races")

In [0]:
spark.sql("select * from races").show()

+------+----+-----+---------+--------------------+----------+--------+--------------------+
|raceId|year|round|circuitId|                name|      date|    time|                 url|
+------+----+-----+---------+--------------------+----------+--------+--------------------+
|     1|2009|    1|        1|Australian Grand ...|2009-03-29|06:00:00|http://en.wikiped...|
|     2|2009|    2|        2|Malaysian Grand Prix|2009-04-05|09:00:00|http://en.wikiped...|
|     3|2009|    3|       17|  Chinese Grand Prix|2009-04-19|07:00:00|http://en.wikiped...|
|     4|2009|    4|        3|  Bahrain Grand Prix|2009-04-26|12:00:00|http://en.wikiped...|
|     5|2009|    5|        4|  Spanish Grand Prix|2009-05-10|12:00:00|http://en.wikiped...|
|     6|2009|    6|        6|   Monaco Grand Prix|2009-05-24|12:00:00|http://en.wikiped...|
|     7|2009|    7|        5|  Turkish Grand Prix|2009-06-07|12:00:00|http://en.wikiped...|
|     8|2009|    8|        9|  British Grand Prix|2009-06-21|12:00:00|http://en.

In [0]:
df_circuits.createOrReplaceTempView("circuits")

In [0]:
spark.sql("select * from circuits").show()

+----------+--------------+--------------------+------------+---------+--------+---------+--------+--------------------+
|circuit_id|   circuit_ref|                Name|    Location|  Country|Latitude|Longitude|Altitude|                 URL|
+----------+--------------+--------------------+------------+---------+--------+---------+--------+--------------------+
|         1|   albert_park|Albert Park Grand...|   Melbourne|Australia|-37.8497|  144.968|      10|http://en.wikiped...|
|         2|        sepang|Sepang Internatio...|Kuala Lumpur| Malaysia| 2.76083|  101.738|      18|http://en.wikiped...|
|         3|       bahrain|Bahrain Internati...|      Sakhir|  Bahrain| 26.0325|  50.5106|       7|http://en.wikiped...|
|         4|     catalunya|Circuit de Barcel...|    Montmeló|    Spain|   41.57|  2.26111|     109|http://en.wikiped...|
|         5|      istanbul|       Istanbul Park|    Istanbul|   Turkey| 40.9517|   29.405|     130|http://en.wikiped...|
|         6|        monaco|   Ci

In [0]:
df_constructors.createOrReplaceTempView("constructors")

In [0]:
spark.sql("select * from constructors").show()

+-------------+--------------+-----------+-----------+--------------------+
|constructorId|constructorRef|       name|nationality|                 url|
+-------------+--------------+-----------+-----------+--------------------+
|            1|       mclaren|    McLaren|    British|http://en.wikiped...|
|            2|    bmw_sauber| BMW Sauber|     German|http://en.wikiped...|
|            3|      williams|   Williams|    British|http://en.wikiped...|
|            4|       renault|    Renault|     French|http://en.wikiped...|
|            5|    toro_rosso| Toro Rosso|    Italian|http://en.wikiped...|
|            6|       ferrari|    Ferrari|    Italian|http://en.wikiped...|
|            7|        toyota|     Toyota|   Japanese|http://en.wikiped...|
|            8|   super_aguri|Super Aguri|   Japanese|http://en.wikiped...|
|            9|      red_bull|   Red Bull|   Austrian|http://en.wikiped...|
|           10|   force_india|Force India|     Indian|http://en.wikiped...|
|           

In [0]:
df_results.createOrReplaceTempView("results")


In [0]:
spark.sql("select * from results").show()

+-------------+--------+----------+---------------+--------------+----+----+------------+------+------+--------+-------------+------------+------+----+--------+--------+-----------+
|constructorId|driverId|fastestLap|fastestLapSpeed|fastestLapTime|grid|laps|milliseconds|number|points|position|positionOrder|positionText|raceId|rank|resultId|statusId|       time|
+-------------+--------+----------+---------------+--------------+----+----+------------+------+------+--------+-------------+------------+------+----+--------+--------+-----------+
|            9|     830|        60|         227.96|      1:17.524|   3|  63|     7354598|    33|    25|       1|            1|           1|  1053|   2|   24986|       1|2:02:34.598|
|          131|       1|        60|        230.403|      1:16.702|   1|  63|     7376598|    44|    19|       2|            2|           2|  1053|   1|   24987|       1|    +22.000|
|            1|     846|        63|        225.819|      1:18.259|   7|  63|     7378300| 

In [0]:
df_races_circuits.createOrReplaceTempView("races_circuits")

In [0]:
spark.sql("select * from races_circuits").show()

+------+----+-----+---------+--------------------+----------+--------+--------------------+----------+-----------+--------------------+------------+---------+--------+---------+--------+--------------------+
|raceId|year|round|circuitId|                name|      date|    time|                 url|circuit_id|circuit_ref|                Name|    Location|  Country|Latitude|Longitude|Altitude|                 URL|
+------+----+-----+---------+--------------------+----------+--------+--------------------+----------+-----------+--------------------+------------+---------+--------+---------+--------+--------------------+
|     1|2009|    1|        1|Australian Grand ...|2009-03-29|06:00:00|http://en.wikiped...|         1|albert_park|Albert Park Grand...|   Melbourne|Australia|-37.8497|  144.968|      10|http://en.wikiped...|
|     2|2009|    2|        2|Malaysian Grand Prix|2009-04-05|09:00:00|http://en.wikiped...|         2|     sepang|Sepang Internatio...|Kuala Lumpur| Malaysia| 2.76083| 

In [0]:
# Performing the analysis on data

In [0]:
# Counting the number of races per year
spark.sql("SELECT year, COUNT(*) AS num_races FROM races GROUP BY year ORDER BY year").show()

+----+---------+
|year|num_races|
+----+---------+
|1950|        7|
|1951|        8|
|1952|        8|
|1953|        9|
|1954|        9|
|1955|        7|
|1956|        8|
|1957|        8|
|1958|       11|
|1959|        9|
|1960|       10|
|1961|        8|
|1962|        9|
|1963|       10|
|1964|       10|
|1965|       10|
|1966|        9|
|1967|       11|
|1968|       12|
|1969|       11|
+----+---------+
only showing top 20 rows



In [0]:
# finding number of races per circuit
spark.sql("SELECT circuitId, COUNT(*) AS num_races FROM races GROUP BY circuitId ORDER BY num_races DESC").show()

+---------+---------+
|circuitId|num_races|
+---------+---------+
|       14|       71|
|        6|       67|
|        9|       56|
|       13|       54|
|       20|       41|
|        7|       40|
|       18|       38|
|       10|       37|
|       11|       36|
|       22|       32|
|        4|       31|
|       39|       31|
|       21|       29|
|        1|       25|
|       23|       25|
|       32|       21|
|       25|       20|
|       30|       20|
|       46|       20|
|       19|       19|
+---------+---------+
only showing top 20 rows



In [0]:
# Calculating average altitude of circuits per country
spark.sql("SELECT Country, AVG(Altitude) AS avg_altitude FROM circuits GROUP BY country ORDER BY avg_altitude DESC").show()

+------------+------------------+
|     Country|      avg_altitude|
+------------+------------------+
|      Mexico|            2227.0|
|      Brazil|             955.5|
|South Africa|             737.5|
|     Austria| 677.3333333333334|
| Switzerland|             551.0|
|      France|             310.0|
|       Japan|             298.0|
|     Hungary|             264.0|
|     Germany|244.66666666666666|
|         USA|             243.0|
|       India|             194.0|
|     Belgium|             192.0|
|      Canada|186.33333333333334|
|       Spain|153.83333333333334|
|      Sweden|             153.0|
|       Italy|            145.75|
|      Turkey|             130.0|
|    Portugal|             106.0|
|          UK|             101.5|
|   Australia|              34.0|
+------------+------------------+
only showing top 20 rows



In [0]:
df_drivers_json.createOrReplaceTempView("drivers")

In [0]:
spark.sql("select * from drivers").show()

+----+----------+--------+----------+--------------------+-----------+------+--------------------+
|code|       dob|driverId| driverRef|                name|nationality|number|                 url|
+----+----------+--------+----------+--------------------+-----------+------+--------------------+
| HAM|1985-01-07|       1|  hamilton|   {Lewis, Hamilton}|    British|    44|http://en.wikiped...|
| HEI|1977-05-10|       2|  heidfeld|    {Nick, Heidfeld}|     German|    \N|http://en.wikiped...|
| ROS|1985-06-27|       3|   rosberg|     {Nico, Rosberg}|     German|     6|http://en.wikiped...|
| ALO|1981-07-29|       4|    alonso|  {Fernando, Alonso}|    Spanish|    14|http://en.wikiped...|
| KOV|1981-10-19|       5|kovalainen|{Heikki, Kovalainen}|    Finnish|    \N|http://en.wikiped...|
| NAK|1985-01-11|       6|  nakajima|  {Kazuki, Nakajima}|   Japanese|    \N|http://en.wikiped...|
| BOU|1979-02-28|       7|  bourdais|{Sébastien, Bourd...|     French|    \N|http://en.wikiped...|
| RAI|1979

In [0]:
# finding number of drivers from each nationality
spark.sql("SELECT nationality, COUNT(*) AS num_drivers FROM drivers GROUP BY nationality ORDER BY num_drivers DESC").show()

+-------------+-----------+
|  nationality|num_drivers|
+-------------+-----------+
|      British|        165|
|     American|        157|
|      Italian|         99|
|       French|         73|
|       German|         50|
|    Brazilian|         32|
|    Argentine|         24|
|        Swiss|         23|
|South African|         23|
|      Belgian|         23|
|     Japanese|         20|
|   Australian|         17|
|        Dutch|         17|
|      Spanish|         15|
|     Austrian|         15|
|     Canadian|         14|
|      Swedish|         10|
|      Finnish|          9|
|New Zealander|          9|
|      Mexican|          6|
+-------------+-----------+
only showing top 20 rows



In [0]:
df_results_json.createOrReplaceTempView("results")

In [0]:
spark.sql("select * from results").show()

+-------------+--------+----------+---------------+--------------+----+----+------------+------+------+--------+-------------+------------+------+----+--------+--------+-----------+
|constructorId|driverId|fastestLap|fastestLapSpeed|fastestLapTime|grid|laps|milliseconds|number|points|position|positionOrder|positionText|raceId|rank|resultId|statusId|       time|
+-------------+--------+----------+---------------+--------------+----+----+------------+------+------+--------+-------------+------------+------+----+--------+--------+-----------+
|            9|     830|        60|         227.96|      1:17.524|   3|  63|     7354598|    33|    25|       1|            1|           1|  1053|   2|   24986|       1|2:02:34.598|
|          131|       1|        60|        230.403|      1:16.702|   1|  63|     7376598|    44|    19|       2|            2|           2|  1053|   1|   24987|       1|    +22.000|
|            1|     846|        63|        225.819|      1:18.259|   7|  63|     7378300| 

In [0]:
# List out races where the winner finished more than 20 seconds ahead of the second-place finisher
spark.sql("SELECT r1.raceId, r1.driverId AS winner_driverId, r1.position, r2.driverId AS second_place_driverId, r2.position, r1.time, r2.time FROM results r1 JOIN results r2 ON r1.raceId = r2.raceId WHERE r1.positionOrder = 1 AND r2.positionOrder = 2 AND (r2.milliseconds - r1.milliseconds) > 20000").show()

+------+---------------+--------+---------------------+--------+-----------+-------+
|raceId|winner_driverId|position|second_place_driverId|position|       time|   time|
+------+---------------+--------+---------------------+--------+-----------+-------+
|  1053|            830|       1|                    1|       2|2:02:34.598|+22.000|
+------+---------------+--------+---------------------+--------+-----------+-------+



In [0]:
spark.sql(" select * from results join drivers where results.driverId == drivers.driverId").show()

+-------------+--------+----------+---------------+--------------+----+----+------------+------+------+--------+-------------+------------+------+----+--------+--------+-----------+----+----------+--------+---------------+--------------------+-----------+------+--------------------+
|constructorId|driverId|fastestLap|fastestLapSpeed|fastestLapTime|grid|laps|milliseconds|number|points|position|positionOrder|positionText|raceId|rank|resultId|statusId|       time|code|       dob|driverId|      driverRef|                name|nationality|number|                 url|
+-------------+--------+----------+---------------+--------------+----+----+------------+------+------+--------+-------------+------------+------+----+--------+--------+-----------+----+----------+--------+---------------+--------------------+-----------+------+--------------------+
|          131|       1|        60|        230.403|      1:16.702|   1|  63|     7376598|    44|    19|       2|            2|           2|  1053|  